In [ ]:
from flask import Flask, request, jsonify
import boto3
import time

app = Flask(__name__)

def runRemoteShellCommands(aws_access_key, aws_secret_key, instance_id, region='us-east-1'):
    # Initialize the session and SSM client
    session = boto3.Session(
        aws_access_key_id=aws_access_key,
        aws_secret_access_key=aws_secret_key,
        region_name=region
    )
    ssm_client = session.client('ssm')
    ec2_client = session.client('ec2')  # EC2 client to fetch instance details

    # Commands to set up Nginx and deploy the React app
    commands = ['''

# Update and install necessary packages
sudo yum update -y

# Install Nginx
sudo yum install nginx -y
sudo systemctl start nginx
sudo systemctl enable nginx

# Install Node.js and npm
curl -sL https://rpm.nodesource.com/setup_16.x | sudo bash -
sudo yum install -y nodejs

# Install Git to clone the Next.js app repository
sudo yum install git -y

# Clone the Next.js app repository (replace with your own repository if needed)
git clone https://github.com/aryamanpathak2022/Fitzone-Gym-By-Promptflow.git
cd Fitzone-Gym-By-Promptflow
cd webapp

# Install the dependencies and build the Next.js app
npm install
npm run build

# Move the built files to the Nginx public directory
sudo mv out/* /usr/share/nginx/html/
# Configure Nginx to serve the built React frontend
sudo bash -c 'cat > /etc/nginx/nginx.conf << EOL
user nginx;
worker_processes auto;
error_log /var/log/nginx/error.log;
pid /run/nginx.pid;

events {
    worker_connections 1024;
}

http {
    include /etc/nginx/mime.types;
    default_type application/octet-stream;

    access_log /var/log/nginx/access.log;

    sendfile on;
    keepalive_timeout 65;

    server {
        listen 80;
        server_name localhost;

        location / {
            root /usr/share/nginx/html;
            index index.html;
            try_files \$uri /index.html;
        }

        location /_next/static/ {
            alias /usr/share/nginx/html/_next/static/;
        }

        location /public/ {
            root /usr/share/nginx/html;
        }

    }
}
EOL'

# Test Nginx configuration for errors
sudo nginx -t

# Restart Nginx to apply the new configuration
sudo systemctl restart nginx

    ''']

    # Send SSM command to the EC2 instance
    response = ssm_client.send_command(
        InstanceIds=[instance_id],
        DocumentName="AWS-RunShellScript",
        Parameters={'commands': commands},
        TimeoutSeconds=600
    )

    command_id = response['Command']['CommandId']
    time.sleep(10)  # Wait to allow command execution

    # Check for command execution status and output
    output = ssm_client.get_command_invocation(
        CommandId=command_id,
        InstanceId=instance_id,
        PluginName='aws:RunShellScript'
    )

    while output['Status'] == "InProgress":
        time.sleep(5)
        output = ssm_client.get_command_invocation(
            CommandId=command_id,
            InstanceId=instance_id,
            PluginName='aws:RunShellScript'
        )
    
    # Fetch the public IP or public DNS of the EC2 instance
    instance_details = ec2_client.describe_instances(InstanceIds=[instance_id])
    public_ip = instance_details['Reservations'][0]['Instances'][0].get('PublicIpAddress')
    public_dns = instance_details['Reservations'][0]['Instances'][0].get('PublicDnsName')

    # Prepare the output
    output_result = {
        'StandardOutputContent': output['StandardOutputContent'],
        'PublicIP': public_ip+":3000",
        'PublicDNS': public_dns,
    }
    
    return output_result

@app.route('/deploy', methods=['GET'])
def deploy():
    # Extract parameters from the GET request
    aws_access_key = request.args.get('aws_access_key')
    aws_secret_key = request.args.get('aws_secret_key')
    instance_id = request.args.get('instance_id')
    region = request.args.get('region', 'us-east-1')  # Default to 'us-east-1' if not provided

    # Check if required parameters are present
    if not aws_access_key or not aws_secret_key or not instance_id:
        return jsonify({'error': 'Missing required parameters'}), 400

    try:
        # Run the shell commands remotely
        result = runRemoteShellCommands(aws_access_key, aws_secret_key, instance_id, region)
        return jsonify(result)
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True)


In [1]:
!aws configure

^C


In [ ]:
pip install boto3

In [ ]:
import boto3
import time

def execute_command_on_ec2(command: str, instance_id: str) -> str:
    """
    Executes a given shell command on a specified EC2 instance using AWS Systems Manager (SSM).

    Args:
        command (str): The command to run on the EC2 instance (e.g., 'sudo yum install -y httpd').
        instance_id (str): The ID of the EC2 instance where the command should be executed.

    Returns:
        str: Output from the command executed on the EC2 instance.
    """

    ssm_client = boto3.client('ssm',region_name='us-east-1')

    # Run the command on the EC2 instance using SSM
    response = ssm_client.send_command(
        InstanceIds=[instance_id],
        DocumentName="AWS-RunShellScript",
        Parameters={'commands': [command]}
    )

    # Fetch the command ID from the response
    command_id = response['Command']['CommandId']

    # Wait for the command to finish execution
    time.sleep(2)  # Small delay to allow the command to start

    # Poll for command output
    output = ssm_client.get_command_invocation(
        CommandId=command_id,
        InstanceId=instance_id
    )

    # Return command output (standard output or errors)
    if output['Status'] == 'Success':
        return output['StandardOutputContent']
    else:
        return output['StandardErrorContent']


instance_id = ''  # Replace with your EC2 instance ID
command = 'sudo yum install -y httpd && sudo systemctl start httpd'

output = execute_command_on_ec2(command, instance_id)
print(output)



In [ ]:
!aws configure list

In [ ]:
!curl http://169.254.169.254/latest/meta-data/instance-id

In [ ]:
import boto3
import time
import os
import json


# Initialize boto3 clients
ec2_client = boto3.client('ec2')
iam_client = boto3.client('iam')
ssm_client = boto3.client('ssm')

def create_key_pair(key_name: str):
    """
    Create an EC2 key pair and save the private key locally.
    """
    try:
        response = ec2_client.create_key_pair(KeyName=key_name)
        private_key = response['KeyMaterial']

        # Save the private key to a file
        with open(f"{key_name}.pem", 'w') as file:
            file.write(private_key)

        os.chmod(f"{key_name}.pem", 0o400)
        print(f"Key pair created and saved as {key_name}.pem")
    except Exception as e:
        print(f"Key pair creation failed: {str(e)}")


def create_iam_role(role_name: str):
    """
    Create an IAM role with SSM permissions and attach the necessary policy.
    """
    try:
        assume_role_policy = {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Principal": {
                        "Service": "ec2.amazonaws.com"
                    },
                    "Action": "sts:AssumeRole"
                }
            ]
        }

        # Create IAM Role
        iam_client.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(assume_role_policy)
        )

        # Attach the AmazonSSMManagedInstanceCore policy
        iam_client.attach_role_policy(
            RoleName=role_name,
            PolicyArn='arn:aws:iam::aws:policy/AmazonSSMManagedInstanceCore'
        )

        print(f"IAM role '{role_name}' created and AmazonSSMManagedInstanceCore policy attached.")

    except Exception as e:
        print(f"IAM role creation failed: {str(e)}")

def create_iam_instance_profile(role_name: str, instance_profile_name: str):
    """
    Create an IAM instance profile and attach the IAM role to it.
    """
    try:
        # Create the instance profile
        iam_client.create_instance_profile(InstanceProfileName=instance_profile_name)

        # Add the role to the instance profile
        iam_client.add_role_to_instance_profile(
            InstanceProfileName=instance_profile_name,
            RoleName=role_name
        )

        print(f"IAM instance profile '{instance_profile_name}' created and role '{role_name}' attached.")

    except Exception as e:
        print(f"IAM instance profile creation failed: {str(e)}")



def create_ec2_instance(key_name: str, instance_profile_name: str):
    """
    Launch an EC2 instance with the specified key pair and IAM instance profile.
    """
    try:
        instance = ec2_client.run_instances(
            ImageId='',  # Amazon Linux 2 AMI
            InstanceType='',
            KeyName=key_name,
            MinCount=1,
            MaxCount=1,
            IamInstanceProfile={
                'Name': instance_profile_name  # Use the instance profile name here
            },
            TagSpecifications=[
                {
                    'ResourceType': 'instance',
                    'Tags': [{'Key': 'Name', 'Value': 'MyWebServer'}]
                }
            ],
            SecurityGroups=['default'],
        )

        instance_id = instance['Instances'][0]['InstanceId']
        print(f"EC2 instance launched with ID: {instance_id}")

        # Wait for instance to be running
        ec2_client.get_waiter('instance_running').wait(InstanceIds=[instance_id])

        # Fetch the public IP
        instance_description = ec2_client.describe_instances(InstanceIds=[instance_id])
        public_ip = instance_description['Reservations'][0]['Instances'][0]['PublicIpAddress']
        print(f"EC2 instance is running. Public IP: {public_ip}")

        return instance_id, public_ip

    except Exception as e:
        print(f"Failed to create EC2 instance: {str(e)}")
        return None,None


def create_security_group():
    """
    Create a security group to allow HTTP and SSH access to the EC2 instance.
    """
    try:
        response = ec2_client.create_security_group(
            Description='Allow SSH and HTTP access',
            GroupName='WebServerSecurityGroup',
            VpcId='vpc-id'  # Replace with your VPC ID
        )

        security_group_id = response['GroupId']

        # Allow SSH (port 22) and HTTP (port 80)
        ec2_client.authorize_security_group_ingress(
            GroupId=security_group_id,
            IpPermissions=[
                {
                    'IpProtocol': 'tcp',
                    'FromPort': 22,
                    'ToPort': 22,
                    'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
                },
                {
                    'IpProtocol': 'tcp',
                    'FromPort': 80,
                    'ToPort': 80,
                    'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
                }
            ]
        )
        print(f"Security group created with ID: {security_group_id}")

        return security_group_id

    except Exception as e:
        print(f"Failed to create security group: {str(e)}")


def execute_command_on_ec2(instance_id: str, command: str):
    """
    Executes a given shell command on a specified EC2 instance using AWS Systems Manager (SSM).
    """
    try:
        response = ssm_client.send_command(
            InstanceIds=[instance_id],
            DocumentName="AWS-RunShellScript",
            Parameters={'commands': [command]}
        )

        command_id = response['Command']['CommandId']

        # Wait a bit for the command to execute
        time.sleep(2)

        output = ssm_client.get_command_invocation(
            CommandId=command_id,
            InstanceId=instance_id
        )

        if output['Status'] == 'Success':
            return output['StandardOutputContent']
        else:
            return output['StandardErrorContent']

    except Exception as e:
        print(f"Error executing command on EC2 instance: {str(e)}")
        return None


def deploy_website(instance_id: str):
    """
    Deploy a basic website by installing Apache and setting up an HTML file.
    """
    # Install Apache
    install_apache_cmd = "sudo yum install -y httpd && sudo systemctl start httpd && sudo systemctl enable httpd"
    execute_command_on_ec2(instance_id, install_apache_cmd)

    # Upload basic website
    html_content = """<html><body><h1>Welcome to My Website!</h1></body></html>"""
    deploy_website_cmd = f"echo '{html_content}' | sudo tee /var/www/html/index.html"
    execute_command_on_ec2(instance_id, deploy_website_cmd)

    print("Website deployed successfully.")


if __name__ == "__main__":
    key_name = ""
    role_name = ""
    instance_profile_name = ""

    # 1. Create EC2 Key Pair
    create_key_pair(key_name)

    # 2. Create IAM Role for SSM
    create_iam_role(role_name)

    # 3. Create IAM Instance Profile
    create_iam_instance_profile(role_name, instance_profile_name)

# 4. Launch EC2 Instance using the instance profile
    instance_id, public_ip = create_ec2_instance(key_name, instance_profile_name)

    # 4. Deploy Website
    deploy_website(instance_id)

    print(f"Website deployed at http://{public_ip}")


In [ ]:
import boto3
import time

def run_command_on_aws(command, instance_ids):
    ssm_client = boto3.client('ssm', region_name='us-east-1')  # Set your region

    try:
        response = ssm_client.send_command(
            InstanceIds=instance_ids,
            DocumentName='AWS-RunShellScript',  # Change as needed
            Parameters={
                'commands': [command]
            },
            TimeoutSeconds=600
        )

        command_id = response['Command']['CommandId']
        print(f'Command sent successfully. Command ID: {command_id}')

        # Wait for the command to finish
        time.sleep(5)  # Adjust based on command duration

        # Get command invocation results
        invocation_response = ssm_client.list_command_invocations(
            CommandId=command_id,
            Details=True
        )

        for invocation in invocation_response['CommandInvocations']:
            for result in invocation['CommandPlugins']:
                print(f"Output of {invocation['InstanceId']}:\n{result['Output']}")

    except Exception as e:
        print(f'Error sending command: {e}')

# Usage example
command_to_run = 'sudo apt update'  # Replace with your command
instance_ids = ['']  # Replace with your instance IDs

run_command_on_aws(command_to_run, instance_ids)


In [ ]:
!ssh -i "keypair"ec2-user@dns_name

In [ ]:
#!/bin/bash

# Variables
KEY_PATH=""  # Path to your EC2 key pair
EC2_USER=""      # EC2 username (change for Ubuntu)
EC2_HOST=""  # EC2 public DNS
LOCAL_BUILD_DIR=""  # Path to the build folder of your React app
REMOTE_APP_DIR=""  # Remote directory to store the app on EC2

# Step 1: Connect to EC2 instance and setup environment
echo "Connecting to EC2 instance..."
ssh -i "$KEY_PATH" "$EC2_USER@$EC2_HOST" << 'ENDSSH'

# Update package manager
echo "Updating packages..."
sudo yum update -y

# Install Node.js
echo "Installing Node.js and npm..."
sudo amazon-linux-extras install nodejs -y

# Install Serve to serve the React app
echo "Installing serve..."
sudo npm install -g serve

# Create a directory for the React app
echo "Creating app directory..."
mkdir -p $REMOTE_APP_DIR

ENDSSH

# Step 2: Transfer React app's build folder to the EC2 instance
echo "Transferring React app build folder to EC2 instance..."
scp -i "$KEY_PATH" -r "$LOCAL_BUILD_DIR" "$EC2_USER@$EC2_HOST:$REMOTE_APP_DIR"

# Step 3: Serve the React app using the 'serve' package
echo "Starting the React app on EC2..."
ssh -i "$KEY_PATH" "$EC2_USER@$EC2_HOST" << 'ENDSSH'

# Navigate to the app directory
cd $REMOTE_APP_DIR

# Serve the app on port 3000
serve -s build -l 3000 &

ENDSSH

echo "Deployment complete. Access your app at http://$EC2_HOST:3000"


In [ ]:
!ssh -i "id_rsa" ubuntu@ec2-54-172-150-249.compute-1.amazonaws.com

In [ ]:
import boto3
import os
import json
import time


## Create IAM role policy
# def CreateInstanceProfileRole():
#     client = boto3.client('iam')
#     assume_role_policy_document = json.dumps({ "Version": "2012-10-17","Statement": [{ "Effect": "Allow", "Principal": {"Service": "ec2.amazonaws.com"},"Action": "sts:AssumeRole"}
#         ]
#     })

#     ## Create IAM role
#     create_role_response = client.create_role(RoleName = "my-instance-role", AssumeRolePolicyDocument = assume_role_policy_document)
#     print (create_role_response)

#     ## Attach policies to above role
#     roleSSMpolicyAttachResponse = client.attach_role_policy( RoleName='my-instance-role', PolicyArn='arn:aws:iam::aws:policy/AmazonSSMFullAccess')
#     print (roleSSMpolicyAttachResponse)

#     ## Attach policies to above role
#     roleS3policyAttachResponse = client.attach_role_policy( RoleName='my-instance-role', PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess')
#     print (roleS3policyAttachResponse)

#     ## Create instance profile
#     instance_profile = client.create_instance_profile ( InstanceProfileName ='Test-instance-profile')
#     print (instance_profile)
#     ## Add roles
#     response = client.add_role_to_instance_profile ( InstanceProfileName = 'Test-instance-profile', RoleName = 'my-instance-role')
#     print (response)

# ## Create EC2 key pair
# def create_key_pair():
#     print("I am inside key pair generation..")
#     ec2_client = boto3.client("ec2", region_name="us-east-1")
#     key_pair = ec2_client.create_key_pair(KeyName="ec2-key-pair")

#     private_key = key_pair["KeyMaterial"]

#     ## write private key to file with 400 permissions
#     with os.fdopen(os.open("F:/RekhuAll/AWS/PythonAllAWS/aws_ec2_key.pem", os.O_WRONLY | os.O_CREAT, 0o400), "w+") as handle: handle.write(private_key)

# ## Create VPC
# def create_aws_vpc():
#     ec2_client = boto3.client("ec2", region_name="us-east-1")
#     vpc = ec2_client.create_vpc( CidrBlock='172.16.0.0/16' )
#     print ("Successfully created vpc details are -  {}".format(vpc))
#     subnet = ec2_client.create_subnet(CidrBlock = '172.16.2.0/24', VpcId= vpc['Vpc']['VpcId'])
#     print("Successfully created subnet details are -  {}".format(subnet))
#     return subnet['Subnet']['SubnetId']


# ## EC2 instance boto3
# def create_instance(subnet):
#     print("Create AWS instances..")
#     ec2_client = boto3.client("ec2", region_name="us-east-1")
#     instances = ec2_client.run_instances(ImageId="ami-048f6ed62451373d9", MinCount=1, MaxCount=1, InstanceType="t2.micro",KeyName="ec2-key-pair", NetworkInterfaces=[{'DeviceIndex': 0,'SubnetId' : subnet, 'AssociatePublicIpAddress': True,}], IamInstanceProfile={'Name': "Test-instance-profile"} )
#     return instances["Instances"][0]["InstanceId"]

# ## How to get the public IP for a running EC2 instance
# def get_public_ip(instance_id):
#     ec2_client = boto3.client("ec2", region_name="us-east-1")
#     reservations = ec2_client.describe_instances(InstanceIds=[instance_id]).get("Reservations")

#     for reservation in reservations:
#         for instance in reservation['Instances']:
#             print(instance.get("PublicIpAddress"))

# ## How to list all running EC2 instances
# def get_running_instances():
#     ec2_client = boto3.client("ec2", region_name="us-east-1")
#     reservations = ec2_client.describe_instances(Filters=[
#         {
#             "Name": "instance-state-name",
#             "Values": ["running"],
#         }
#     ]).get("Reservations")

#     for reservation in reservations:
#         for instance in reservation["Instances"]:
#             instance_id = instance["InstanceId"]
#             instance_type = instance["InstanceType"]
#             public_ip = instance["PublicIpAddress"]
#             private_ip = instance["PrivateIpAddress"]
#             print(f"{instance_id}, {instance_type}, {public_ip}, {private_ip}")

## Run command against your linux VM
# def runRemoteShellCommands (InstanceId):
#     ssm_client = boto3.client('ssm', region_name="us-east-1")
#     response = ssm_client.send_command( InstanceIds=[InstanceId], DocumentName="AWS-RunShellScript", Parameters={'commands':[ 'sudo systemctl status amazon-ssm-agent']},)
#     command_id = response['Command']['CommandId']
#     output = ssm_client.get_command_invocation( CommandId=command_id, InstanceId=InstanceId)
#     while output['Status'] == "InProgress":
#         output = ssm_client.get_command_invocation( CommandId=command_id, InstanceId=InstanceId)
#     print(output['StandardOutputContent'])

# #CreateInstanceProfileRole()

# #create_key_pair()

# #returnedsubnetid  = create_aws_vpc()

# #createdinstanceID = create_instance(returnedsubnetid)

# get_running_instances()

# get_public_ip("i-01bd74c9cfb67a532")

# runRemoteShellCommands("i-01bd74c9cfb67a532")


def runRemoteShellCommands (aws_access_key, aws_secret_key, instance_id, region='us-east-1'):
    session = boto3.Session(
        aws_access_key_id=aws_access_key,
        aws_secret_access_key=aws_secret_key,
        region_name=region
    )
    ssm_client = session.client('ssm')
    # commands = [
    #     'sudo yum update -y',           # For Amazon Linux
    #     'sudo yum install -y nginx',    # Install Nginx
    #     'sudo systemctl start nginx',   # Start Nginx
    #     'sudo systemctl enable nginx',  # Enable Nginx to start at boot
    #     'sudo systemctl status nginx'   # Check Nginx status
    # ]
    # commands=['echo Hello World']
    commands=['''sudo yum update -y
sudo yum install -y nginx
sudo systemctl start nginx
sudo systemctl enable nginx
sudo systemctl status nginx
sudo yum install -y python3-pip
sudo pip3 install boto3
sudo pip3 install requests
sudo pip3 install Flask
sudo pip3 install flask_cors
sudo pip3 install flask_restful
sudo pip3 install flask_sqlalchemy
sudo pip3 install sqlalchemy
sudo pip3 install psycopg2
sudo pip3 install flask_jwt_extended

''']


    # ssm_client = boto3.client('ssm', region_name="us-east-1") # Updated region to match instance
    # response = ssm_client.send_command( InstanceIds=[InstanceId], DocumentName="AWS-RunShellScript", Parameters={'commands':[
    #     'sudo yum update -y',           # For Amazon Linux
    #     'sudo yum install -y nginx',    # Install Nginx
    #     'sudo systemctl start nginx',   # Start Nginx
    #     'sudo systemctl enable nginx',  # Enable Nginx to start at boot
    #     'sudo systemctl status nginx'   # Check Nginx status
    # ]},)
    response = ssm_client.send_command(
        InstanceIds=[instance_id],
        DocumentName="AWS-RunShellScript",
        Parameters={'commands': commands}
    )
    print(response)
    command_id = response['Command']['CommandId']
    print(f"Command ID: {command_id}")
    time.sleep(5)
    # output = ssm_client.get_command_invocation( CommandId=command_id, InstanceId=InstanceId,PluginName='aws:RunShellScript')
    output = ssm_client.get_command_invocation(
        CommandId=command_id,
        InstanceId=instance_id,
        PluginName='aws:RunShellScript'
    )

    while output['Status'] == "InProgress":
        # time.sleep(2)  # Wait before checking the status again
        output = ssm_client.get_command_invocation(
            CommandId=command_id,
            InstanceId=instance_id,
            PluginName='aws:RunShellScript'
        )
        print(output)
    print(output['StandardOutputContent'])

# Ensure the instance ID is correct and the instance is in a running state
# get_running_instances()

# # Verify the region matches the instance's region
# get_public_ip("i-01bd74c9cfb67a532")

# If the instance is running and the region is correct, try running the command again
runRemoteShellCommands("","","")



In [ ]:
pip install boto3

In [ ]:
!sudo apt install curl
!curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.39.7/install.sh | bash
!source ~/.bashrc
!nvm install node

In [ ]:
!source ~/.bashrc



In [ ]:
# Update and install necessary packages
sudo yum update -y

# Install Nginx
sudo yum install nginx -y
sudo systemctl start nginx
sudo systemctl enable nginx

# Install Node.js and npm
curl -sL https://rpm.nodesource.com/setup_16.x | sudo bash -
sudo yum install -y nodejs

# Install Git to clone the Next.js app repository
sudo yum install git -y

# Clone the Next.js app repository (replace with your own repository if needed)

git clone app.git
cd app
cd webapp

# Install the dependencies and build the Next.js app
npm install
npm run build
# Export the Next.js app as static files
# npm run export

# Move the exported files to the Nginx public directory
# sudo mv out/* /usr/share/nginx/html/
# Configure Nginx to serve the exported Next.js frontend
sudo bash -c 'cat > /etc/nginx/nginx.conf << EOL
user nginx;
worker_processes auto;
error_log /var/log/nginx/error.log;
pid /run/nginx.pid;

events {
    worker_connections 1024;
}

http {
    include /etc/nginx/mime.types;
    default_type application/octet-stream;

    access_log /var/log/nginx/access.log;

    sendfile on;
    keepalive_timeout 65;

    server {
        listen 80;
        server_name localhost;

        location / {
            root /usr/share/nginx/html;
            index index.html;
            try_files \$uri /index.html;
        }

        location /_next/static/ {
            alias /usr/share/nginx/html/_next/static/;
        }

        location /public/ {
            root /usr/share/nginx/html;
        }

    }
    server {
    listen 80;
    server_name ec2-18-209-159-55.compute-1.amazonaws.com;

    location / {
        proxy_pass http://localhost:3000;
        proxy_http_version 1.1;
        proxy_set_header Upgrade $http_upgrade;
        proxy_set_header Connection 'upgrade';
        proxy_set_header Host $host;
        proxy_cache_bypass $http_upgrade;
    }
}
}
EOL'

# Test Nginx configuration for errors
# sudo nginx -t

# # Restart Nginx to apply the new configuration
# sudo systemctl restart nginx
npm start